In [19]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report
import unicodedata
import pythainlp
import re
import emoji

In [2]:
def normalize_double_quote(text: str):
    all_quote = [
    '“', # U+201c
    '”', # U+201d
    "„", # U+201e
    '«', # U+00AB
    '»', # U+00BB
    '„', # U+201E
    '“', # U+201C
    '‟', # U+201F
    '”', # U+201D
    '❝', # U+275D
    '❞', # U+275E
    '〝', # U+301D
    '〞', # U+301E
    '〟', # U+301F
    '＂', # U+FF02
    ]
    std_quote = "\"" #U+0022
    table = str.maketrans(dict.fromkeys(all_quote, std_quote))
    return text.translate(table)

def normalize_single_quote(text: str):
    all_quote = ['\u02BB', '\u02BC', '\u066C', '\u2018', '\u2019', '\u201A', '\u275B', '\u275C']
    std_quote = "\'"
    table = str.maketrans(dict.fromkeys(all_quote, std_quote))
    return text.translate(table)

def replace_url(text):
    URL_PATTERN = r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}
]+|
[^\s()]*?\)|
)+(?:
[^\s()]*?\)|
|[^\s`!()
{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""
    return re.sub(URL_PATTERN, 'xxurl', text)

def replace_rep(text):
    def _replace_rep(m):
        c,cc = m.groups()
        return f'{c}xxrep'
    re_rep = re.compile(r'(\S)(\1{2,})')
    return re_rep.sub(_replace_rep, text)

def ungroup_emoji(toks):
    res = []
    for tok in toks:
        if emoji.emoji_count(tok) == len(tok):
            for char in tok:
                res.append(char)
        else:
            res.append(tok)
    return res

def preprocess_func(text):
    new_text = unicodedata.normalize('NFKD', text)
    new_text = pythainlp.util.normalize(new_text)
    new_text = normalize_double_quote(new_text)
    new_text = normalize_single_quote(new_text)

    new_text = new_text.lower().strip()
    new_text = replace_url(new_text)
    new_text = replace_rep(new_text)

    res = [word for word in pythainlp.word_tokenize(new_text) if word and not re.search(pattern=r"\s+", string=word)]

    res = ungroup_emoji(res)
    return res

In [4]:
train_df = pd.read_csv('./train_data_fake_news.csv')
val_df = pd.read_csv('./valid_data_fake_news.csv')
test_df = pd.read_csv('./test_data_fake_news.csv')

In [5]:
train_df

,text,label
0,ไก่ลุงเคนกินแล้วมีแต่สารเร่งโต,1
1,เครื่องดื่มต้านCovid-19ค่าพีเอชคือความเป็นด่าง...,1
2,"แค่ฟังชื่อก็ขนลุก!""ต้นป่าช้าเหงา""สมุนไพรชั้นดี...",1
3,พิชัย'แนะ'ประยุทธ์'เพิ่มเซลล์สมองแก้ปัญหาเศรษฐ...,0
4,อุจจาระตกค้างเรื่องใหญ่!แจกสูตรดีท็อกซ์ล้างลำไ...,1
...,...,...
2358,เดี๋ยวนี้มีใบทุเรียนน้ำแกโรคมะเร็งหายขาดจ้า,1
2359,อาหาร8ประเภทที่ไม่ควรนำไปอุ่นซ้ำอีกรอบ|||จะต้อ...,1
2360,เภสัชเผยยากลุ่มNSAIDsห้ามกินร่วมกันเสี่ยงอันตร...,0
2361,สบส.ฟันธง!!วัยทำงานเกือบครึ่งอนาคตเสี่ยงโรคเรื...,0


In [6]:
val_df

,text,label
0,กรมอนามัยแนะวิธีเก็บน้ำฝนลดการปนเปื้อน–ต้มก่อน...,0
1,แนะ!!เข้าใจยอมรับเด็กออทิสติกรักษาเร็วได้ผลเร็...,0
2,3วิธีง่ายๆในการกำจัดเหนียงใต้คางบ๊ายบายคางสองช...,1
3,"แจกฟรีพันธุ์""ต้นอังกาบหนู""สมุนไพรยอดฮิตช่วยรัก...",1
4,สถาบันสุขภาพเด็กแห่งชาติมหาราชินีแนะนำฉีดวัคซี...,0
...,...,...
501,โควิดกับไข้หวัดใหญ่ก็เหมือนกันแต่คนละสายพันธุ์...,1
502,กระบก..มากคุณค่าช่วยให้เจริญอาหารบำรุงไต|||[K]...,1
503,โรคติดเชื้อไวรัสโคโรนา2019CoronavirusDisease20...,0
504,แพทย์แนะ!!หยุดพักเพื่อเลี่ยงอุบัติเหตุจากบ้านห...,0


In [7]:
train_val_df = pd.concat([train_df, val_df]).reset_index().drop(columns='index')

In [8]:
train_val_df

,text,label
0,ไก่ลุงเคนกินแล้วมีแต่สารเร่งโต,1
1,เครื่องดื่มต้านCovid-19ค่าพีเอชคือความเป็นด่าง...,1
2,"แค่ฟังชื่อก็ขนลุก!""ต้นป่าช้าเหงา""สมุนไพรชั้นดี...",1
3,พิชัย'แนะ'ประยุทธ์'เพิ่มเซลล์สมองแก้ปัญหาเศรษฐ...,0
4,อุจจาระตกค้างเรื่องใหญ่!แจกสูตรดีท็อกซ์ล้างลำไ...,1
...,...,...
2864,โควิดกับไข้หวัดใหญ่ก็เหมือนกันแต่คนละสายพันธุ์...,1
2865,กระบก..มากคุณค่าช่วยให้เจริญอาหารบำรุงไต|||[K]...,1
2866,โรคติดเชื้อไวรัสโคโรนา2019CoronavirusDisease20...,0
2867,แพทย์แนะ!!หยุดพักเพื่อเลี่ยงอุบัติเหตุจากบ้านห...,0


In [9]:
tfidf = TfidfVectorizer(tokenizer=preprocess_func, ngram_range=(1,2), min_df=20, sublinear_tf=True)
text_train = tfidf.fit_transform(train_val_df['text'])
text_test = tfidf.transform(test_df['text'])

c:\Users\foofo\Desktop\workspace\fake_news_project\venv\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [10]:
text_train.shape, text_test.shape

((2869, 9391), (507, 9391))

In [11]:
X_train = text_train.toarray()
X_test = text_test.toarray()
X_train.shape, X_test.shape

((2869, 9391), (507, 9391))

In [12]:
y_train = train_val_df['label']
y_test = test_df['label']

In [13]:
model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.8856015779092702

In [13]:
from sklearn.metrics import classification_report
y_val_pred = model.predict(X_test)
print(classification_report(y_test, y_val_pred))

              precision    recall  f1-score   support

           0       0.86      0.87      0.86       212
           1       0.90      0.90      0.90       295

    accuracy                           0.89       507
   macro avg       0.88      0.88      0.88       507
weighted avg       0.89      0.89      0.89       507



In [28]:
model_2 = LogisticRegression()
# model_2 = LogisticRegression(C=2., penalty='l2', solver='liblinear', dual=False, multi_class='ovr')
model_2.fit(X_train, y_train)
model_2.score(X_test, y_test)

0.9171597633136095

In [15]:
y_val_pred_2 = model_2.predict(X_test)
print(classification_report(y_test, y_val_pred_2))

              precision    recall  f1-score   support

           0       0.91      0.90      0.90       212
           1       0.93      0.94      0.93       295

    accuracy                           0.92       507
   macro avg       0.92      0.92      0.92       507
weighted avg       0.92      0.92      0.92       507



In [18]:
model_3 = GaussianNB()
model_3.fit(X_train, y_train)
model_3.score(X_test, y_test)

0.8678500986193294

In [19]:
y_val_pred_3 = model_3.predict(X_test)
print(classification_report(y_test, y_val_pred_3))

              precision    recall  f1-score   support

           0       0.83      0.86      0.85       212
           1       0.90      0.87      0.88       295

    accuracy                           0.87       507
   macro avg       0.86      0.87      0.86       507
weighted avg       0.87      0.87      0.87       507



In [21]:
model_4 = SVC()
model_4.fit(X_train, y_train)
model_4.score(X_test, y_test)

0.9171597633136095

In [22]:
y_val_pred_4 = model_4.predict(X_test)
print(classification_report(y_test, y_val_pred_4))

              precision    recall  f1-score   support

           0       0.91      0.89      0.90       212
           1       0.92      0.94      0.93       295

    accuracy                           0.92       507
   macro avg       0.92      0.91      0.91       507
weighted avg       0.92      0.92      0.92       507



In [23]:
model_5 = RandomForestClassifier()
model_5.fit(X_train, y_train)
model_5.score(X_test, y_test)

0.9112426035502958

In [24]:
y_val_pred_5 = model_5.predict(X_test)
print(classification_report(y_test, y_val_pred_5))

              precision    recall  f1-score   support

           0       0.92      0.86      0.89       212
           1       0.91      0.95      0.93       295

    accuracy                           0.91       507
   macro avg       0.91      0.90      0.91       507
weighted avg       0.91      0.91      0.91       507



Hyperparameter

SVM

In [29]:
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [2, 1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}
svm_grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
svm_grid.fit(X_train, y_train)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV 1/5] END ........C=0.1, gamma=2, kernel=rbf;, score=0.538 total time=  45.7s
[CV 2/5] END ........C=0.1, gamma=2, kernel=rbf;, score=0.537 total time= 1.1min
[CV 3/5] END ........C=0.1, gamma=2, kernel=rbf;, score=0.538 total time=  40.3s
[CV 4/5] END ........C=0.1, gamma=2, kernel=rbf;, score=0.537 total time=  43.5s
[CV 5/5] END ........C=0.1, gamma=2, kernel=rbf;, score=0.538 total time=  43.7s
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.848 total time=  41.3s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.828 total time=  37.2s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.814 total time=  52.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.814 total time=  56.1s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.848 total time=  34.1s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.552 total time=  33.8s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [2, 1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [30]:
svm_grid.best_params_

{'C': 100, 'gamma': 1, 'kernel': 'rbf'}

In [31]:
svm_grid.best_estimator_

SVC(C=100, gamma=1)

In [34]:
svm_grid.score(X_test, y_test)

0.9171597633136095

In [33]:
svm_grid_y_pred = svm_grid.predict(X_test)
print(classification_report(y_test, svm_grid_y_pred))

              precision    recall  f1-score   support

           0       0.90      0.90      0.90       212
           1       0.93      0.93      0.93       295

    accuracy                           0.92       507
   macro avg       0.92      0.91      0.91       507
weighted avg       0.92      0.92      0.92       507



Random Forest

In [39]:
param_grid = {
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'min_samples_leaf': [1, 2, 4],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}
rdf_grid = RandomizedSearchCV(RandomForestClassifier(), param_grid, refit = True, verbose = 3)
rdf_grid.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END max_depth=10, min_samples_leaf=4, n_estimators=2000;, score=0.885 total time=  43.6s
[CV 2/5] END max_depth=10, min_samples_leaf=4, n_estimators=2000;, score=0.855 total time=  47.6s
[CV 3/5] END max_depth=10, min_samples_leaf=4, n_estimators=2000;, score=0.841 total time=  49.6s
[CV 4/5] END max_depth=10, min_samples_leaf=4, n_estimators=2000;, score=0.843 total time=  51.2s
[CV 5/5] END max_depth=10, min_samples_leaf=4, n_estimators=2000;, score=0.853 total time=  50.1s
[CV 1/5] END max_depth=80, min_samples_leaf=4, n_estimators=1000;, score=0.901 total time=  44.3s
[CV 2/5] END max_depth=80, min_samples_leaf=4, n_estimators=1000;, score=0.875 total time=  42.1s
[CV 3/5] END max_depth=80, min_samples_leaf=4, n_estimators=1000;, score=0.866 total time=  39.6s
[CV 4/5] END max_depth=80, min_samples_leaf=4, n_estimators=1000;, score=0.864 total time=  35.3s
[CV 5/5] END max_depth=80, min_samples_leaf=4, n_estimato

RandomizedSearchCV(estimator=RandomForestClassifier(),
                   param_distributions={'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, None],
                                        'min_samples_leaf': [1, 2, 4],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   verbose=3)

In [44]:
rdf_grid.best_params_

{'n_estimators': 400, 'min_samples_leaf': 1, 'max_depth': 90}

In [41]:
rdf_grid.best_estimator_

RandomForestClassifier(max_depth=90, n_estimators=400)

In [42]:
rdf_grid.score(X_test, y_test)

0.9171597633136095

In [43]:
rdf_grid_y_pred = rdf_grid.predict(X_test)
print(classification_report(y_test, rdf_grid_y_pred))

              precision    recall  f1-score   support

           0       0.93      0.87      0.90       212
           1       0.91      0.95      0.93       295

    accuracy                           0.92       507
   macro avg       0.92      0.91      0.91       507
weighted avg       0.92      0.92      0.92       507



In [30]:
param_grid = {
 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
 'solver': ['liblinear'],  
 'multi_class': ['ovr']}
lgt_grid = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose = 3)
lgt_grid.fit(X_train, y_train)

Fitting 5 folds for each of 7 candidates, totalling 35 fits
[CV 1/5] END C=0.001, multi_class=ovr, solver=liblinear;, score=0.538 total time=   0.2s
[CV 2/5] END C=0.001, multi_class=ovr, solver=liblinear;, score=0.537 total time=   0.1s
[CV 3/5] END C=0.001, multi_class=ovr, solver=liblinear;, score=0.537 total time=   0.1s
[CV 4/5] END C=0.001, multi_class=ovr, solver=liblinear;, score=0.537 total time=   0.1s
[CV 5/5] END C=0.001, multi_class=ovr, solver=liblinear;, score=0.538 total time=   0.1s
[CV 1/5] END C=0.01, multi_class=ovr, solver=liblinear;, score=0.685 total time=   0.1s
[CV 2/5] END C=0.01, multi_class=ovr, solver=liblinear;, score=0.667 total time=   0.1s
[CV 3/5] END C=0.01, multi_class=ovr, solver=liblinear;, score=0.660 total time=   0.1s
[CV 4/5] END C=0.01, multi_class=ovr, solver=liblinear;, score=0.657 total time=   0.1s
[CV 5/5] END C=0.01, multi_class=ovr, solver=liblinear;, score=0.677 total time=   0.1s
[CV 1/5] END C=0.1, multi_class=ovr, solver=liblinear;,

GridSearchCV(estimator=LogisticRegression(),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'multi_class': ['ovr'], 'solver': ['liblinear']},
             verbose=3)

In [31]:
lgt_grid.best_params_

{'C': 10, 'multi_class': 'ovr', 'solver': 'liblinear'}

In [32]:
lgt_grid.best_estimator_

LogisticRegression(C=10, multi_class='ovr', solver='liblinear')

In [33]:
lgt_grid.score(X_test, y_test)

0.9211045364891519

In [34]:
lgt_grid_y_pred = lgt_grid.predict(X_test)
print(classification_report(y_test, lgt_grid_y_pred))

              precision    recall  f1-score   support

           0       0.91      0.91      0.91       212
           1       0.93      0.93      0.93       295

    accuracy                           0.92       507
   macro avg       0.92      0.92      0.92       507
weighted avg       0.92      0.92      0.92       507

